Will be done in Part 08 again.

In [113]:
import sqlite3
import urllib
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import spiceypy as sp
from astropy import units as u
from astropy.table import QTable
from astropy.time import Time
from astroquery.jplhorizons import Horizons

warnings.filterwarnings('ignore', append=True)

def download_file(dl_path, dl_url):
    """ Download files from the Internet.

    Parameters
    ----------
    dl_path : str
        Download path on the local machine, relative to this function.
    dl_url : str
        Download url of the requested file.
    """

    # Make directory if it does not exist
    dl_path = Path(dl_path)
    dl_path.mkdir(parents=True, exist_ok=True)
    # Get the file name from the url
    file_name = dl_url.split('/')[-1]

    # If the file is not present in the download directory -> download it
    if not (dl_path/file_name).exists():
        # Download the file with the urllib  package
        urllib.request.urlretrieve(dl_url, dl_path/file_name)

# Load the SPICE kernel meta file
sp.furnsh('kernel_meta.txt')

# The comet data from the Minor Planet Center
download_file(
    'raw_data/',
    'https://www.minorplanetcenter.net/Extended_Files/cometels.json.gz'
)

_, GM_SUN = sp.bodvcd(bodyid=10, item='GM', maxn=1)
GM_SUN = GM_SUN[0]

NOW_UTC = Time.now().utc
NOW_ET = sp.utc2et(NOW_UTC.strftime('%Y-%m-%dT%H:%M:%S'))
print(f"{NOW_UTC} (in SPICE ET: {NOW_ET})")

c_df = pd.read_json('raw_data/cometels.json.gz', compression='gzip')
c_df.tail()

2023-11-23 08:53:33.564773 (in SPICE ET: 754001682.1828934)


,Orbit_type,Provisional_packed_desig,Year_of_perihelion,Month_of_perihelion,Day_of_perihelion,Perihelion_dist,e,Peri,Node,i,Epoch_year,Epoch_month,Epoch_day,H,G,Designation_and_name,Ref,Comet_num
1089,P,NaN,2025,12,8.8163,3.005487,0.307438,43.5395,178.9645,20.1742,2023.0,11.0,22.0,13.5,4.0,469P/PANSTARRS,MPEC 2023,469.0
1090,P,NaN,2023,12,18.2515,2.728714,0.390381,152.0501,246.1830,8.8387,2023.0,11.0,22.0,15.5,4.0,470P/PANSTARRS,MPEC 2023,470.0
1091,P,NaN,2023,12,20.2949,2.123404,0.627962,94.9585,283.3426,4.7909,2023.0,11.0,22.0,12.0,4.0,471P,MPEC 2023,471.0
1092,I,NaN,2017,9,9.4886,0.255240,1.199252,241.6845,24.5997,122.6778,2017.0,9.0,4.0,23.0,2.0,1I/`Oumuamua,MPC107687,1.0
1093,I,NaN,2019,12,8.5548,2.006548,3.356633,209.1251,308.1480,44.0527,2019.0,12.0,23.0,11.0,4.0,2I/Borisov,MPEC 2023,2.0


In [137]:
# First we parse the date and time information. The dataset contains two
# time related information: the date-time of the last perihelion passage and
# another variable called Epoch. However, "epoch" is not related to the mean
# anomaly related epoch and represents other time information in this case.
#
# For our "actual" Epoch case we need to create a UTC time string based on the
# date and time of the last perihelion passage (the time corresponds to a mean
# anomaly of 0 degrees). The Day is given in DAY.FRACTION_OF_DAY. We extract
# only the day
times_utc = []
times_et = []
day = c_df["Day_of_perihelion"].astype(int)
fracday = c_df["Day_of_perihelion"].values % 1

for i, row in c_df.iterrows():
    datetime = Time("{}-{}-{}".format(row["Year_of_perihelion"], row["Month_of_perihelion"], day[i]),
                    format='iso',
                    scale='utc'
                    ) + fracday[i]*u.day
    times_utc.append(datetime.iso)
    times_et.append(sp.utc2et(datetime.iso))
c_df["epoch_utc"] = times_utc
c_df["epoch_et"] = times_et

In [126]:
# Extract Hale-Bopp
hb = c_df.loc[c_df['Designation_and_name'].str.contains('Hale-Bopp')].to_records()

# Set an array with orbital elements in a required format for the conics
# function. Note: the mean anomaly is 0 degrees and will be set as a default
# value in the SQLite database
elt_hb = [
    sp.convrt(hb['Perihelion_dist'][0], 'AU', 'km'),
    hb['e'][0],
    np.deg2rad(hb['i']),
    np.deg2rad(hb['Node']),
    np.deg2rad(hb['Peri']),
    0.0,
    hb['epoch_et'],
    GM_SUN
]

vec_hb = sp.conics(elt_hb, NOW_ET)

hb_hori = QTable(Horizons(id='Hale-Bopp', epochs=NOW_UTC.jd, id_type='id').vectors())
print(f" x, y, z   [MPC & SPICE]: {sp.convrt(vec_hb[0], 'km', 'au')}, {sp.convrt(vec_hb[1], 'km', 'au')}, {sp.convrt(vec_hb[2], 'km', 'au')}")
print(f" x, y, z    [HORIZONS]  : {hb_hori['x'][0]}, {hb_hori['y'][0]}, {hb_hori['z'][0]}")
print(f"vx, vy, vz [MPC & SPICE]: {vec_hb[3]}, {vec_hb[4]}, {vec_hb[5]}")
print(f"vx, vy, vz  [HORIZONS]  : {hb_hori['vx'][0].to_value('km/s')}, {hb_hori['vy'][0].to_value('km/s')}, {hb_hori['vz'][0].to_value('km/s')}")

 x, y, z   [MPC & SPICE]: 4.016139175848223, -20.30116678240435, -43.13519099409667
 x, y, z    [HORIZONS]  : 4.069578002581272 AU, -20.44382192883483 AU, -43.06689712495626 AU
vx, vy, vz [MPC & SPICE]: 0.6343995481034933, -3.106455824579049, -4.693109598088302
vx, vy, vz  [HORIZONS]  : 0.63716877875072, -3.124449446641202, -4.68169119583522


In [132]:
c_df['a'] = c_df['Perihelion_dist'] / (1 - c_df['e'])
c_df['Q'] = c_df["a"]*(1 + c_df["e"])
_open = c_df["e"] > 1
c_df["a"][_open] = np.nan
c_df["Q"][_open] = np.nan

,Orbit_type,Provisional_packed_desig,Year_of_perihelion,Month_of_perihelion,Day_of_perihelion,Perihelion_dist,e,Peri,Node,i,...,Epoch_day,H,G,Designation_and_name,Ref,Comet_num,epoch_utc,epoch_et,a,Q
0,D,H66G010,2024,10,28.9750,0.402760,0.852140,175.6870,93.5420,8.7160,...,22.0,9.0,4.0,D/1766 G1 (Helfenzrieder),"26, 114",NaN,2024-10-28 23:24:00.000,7.834299e+08,2.723928,5.045096
1,D,I84O010,2026,1,7.2809,1.322422,0.571218,339.8250,332.8950,9.3070,...,22.0,9.0,4.0,D/1884 O1 (Barnard),"89, 256",NaN,2026-01-07 06:44:29.760,8.210403e+08,3.084136,4.845850
2,D,I86K010,2025,6,18.2903,1.888337,0.468356,208.4372,39.1628,10.9262,...,22.0,9.0,4.0,D/1886 K1 (Brooks),"89, 256",NaN,2025-06-18 06:58:01.920,8.035020e+08,3.551882,5.215428
3,D,I94F010,2021,12,18.9599,1.171062,0.693883,198.4069,297.3382,1.4229,...,22.0,9.0,4.0,D/1894 F1 (Denning),CCO 8,NaN,2021-12-18 23:02:15.360,6.931406e+08,3.825537,6.480013
4,D,I95Q010,2025,9,5.4923,1.389395,0.626819,199.2589,135.7879,13.5231,...,22.0,9.0,4.0,D/1895 Q1 (Swift),"13, 428",NaN,2025-09-05 11:48:54.720,8.103450e+08,3.723113,6.056831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,P,NaN,2025,12,8.8163,3.005487,0.307438,43.5395,178.9645,20.1742,...,22.0,13.5,4.0,469P/PANSTARRS,MPEC 2023,469.0,2025-12-08 19:35:28.320,8.184946e+08,4.339665,5.673843
1090,P,NaN,2023,12,18.2515,2.728714,0.390381,152.0501,246.1830,8.8387,...,22.0,15.5,4.0,470P/PANSTARRS,MPEC 2023,470.0,2023-12-18 06:02:09.600,7.561514e+08,4.476097,6.223481
1091,P,NaN,2023,12,20.2949,2.123404,0.627962,94.9585,283.3426,4.7909,...,22.0,12.0,4.0,471P,MPEC 2023,471.0,2023-12-20 07:04:39.360,7.563279e+08,5.707492,9.291580
1092,I,NaN,2017,9,9.4886,0.255240,1.199252,241.6845,24.5997,122.6778,...,4.0,23.0,2.0,1I/`Oumuamua,MPC107687,1.0,2017-09-09 11:43:35.040,5.582295e+08,NaN,NaN


In [139]:
cometdir = Path('../_databases/_comets/')
cometdir.mkdir(parents=True, exist_ok=True)

# Create / Connect to a comet database and set the cursor
con = sqlite3.connect(cometdir/'comets.db')
cur = con.cursor()

# Create (if not existing) a comets' main table, where miscellaneous
# parameters are stored
cur.execute('CREATE TABLE IF NOT EXISTS ' \
            'comets_main(NAME TEXT PRIMARY KEY, ' \
                        'ORBIT_TYPE TEXT, ' \
                        'PERIHELION_AU REAL, ' \
                        'SEMI_MAJOR_AXIS_AU REAL, ' \
                        'APHELION_AU REAL, ' \
                        'ECCENTRICITY REAL, ' \
                        'INCLINATION_DEG REAL, ' \
                        'ARG_OF_PERIH_DEG REAL, ' \
                        'LONG_OF_ASC_NODE_DEG REAL, ' \
                        'MEAN_ANOMALY_DEG REAL DEFAULT 0.0, ' \
                        'EPOCH_UTC TEXT, ' \
                        'EPOCH_ET REAL, ' \
                        'ABSOLUTE_MAGNITUDE REAL, ' \
                        'SLOPE_PARAMETER REAL'
                        ')')

# Insert the data
cur.executemany('INSERT OR REPLACE INTO ' \
                'comets_main(NAME, ' \
                            'ORBIT_TYPE, ' \
                            'PERIHELION_AU, ' \
                            'SEMI_MAJOR_AXIS_AU, ' \
                            'APHELION_AU, ' \
                            'ECCENTRICITY, ' \
                            'INCLINATION_DEG, ' \
                            'ARG_OF_PERIH_DEG, ' \
                            'LONG_OF_ASC_NODE_DEG, ' \
                            'EPOCH_UTC, ' \
                            'EPOCH_ET, ' \
                            'ABSOLUTE_MAGNITUDE, ' \
                            'SLOPE_PARAMETER'
                            ') ' \
                'VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', \
                c_df[['Designation_and_name', \
                      'Orbit_type', \
                      'Perihelion_dist', \
                      'a', \
                      'Q', \
                      'e', \
                      'i', \
                      'Peri', \
                      'Node', \
                      'epoch_utc', \
                      'epoch_et', \
                      'H', \
                      'G']].values)

# Commit
con.commit()

# Close the database. The database shall be the fundament for the next
# tutorial sessions
con.close()